In [6]:
pip install scikit-learn==1.2.2

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
df = pd.read_csv("C:/Users/Prem Gupta/Downloads/filtered_cleaned_data_final2.csv")

In [24]:
import gensim
from gensim.models import Word2Vec
import numpy as np

# Tokenize text (split into words)
tokenized_text = [tweet.split() for tweet in df["cleaned_text"]]

# Train Word2Vec model
w2v_model = Word2Vec(sentences=tokenized_text, vector_size=100, window=5, min_count=2, workers=4)

# Function to convert text to vector
def vectorize_text(text, model):
    words = text.split()
    vector = [model.wv[word] for word in words if word in model.wv]
    return sum(vector) / len(vector) if vector else np.zeros(100)

# Convert dataset to vectors
X = np.array([vectorize_text(tweet, w2v_model) for tweet in df["cleaned_text"]])
y = df["label"]  # Labels remain the same



In [25]:
from imblearn.over_sampling import SMOTE
from collections import Counter

smote = SMOTE(sampling_strategy="auto", random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Check new class distribution
print("Resampled dataset distribution:", Counter(y_resampled))


Resampled dataset distribution: Counter({'Anxiety': 15066, 'Depression': 15066, 'Stress': 15066, 'Bipolar': 15066, 'Personality disorder': 15066})


In [26]:
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import accuracy_score, classification_report

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Train LightGBM Model
model = lgb.LGBMClassifier(n_estimators=500, learning_rate=0.05)
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Evaluate model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.054353 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 25500
[LightGBM] [Info] Number of data points in the train set: 60264, number of used features: 100
[LightGBM] [Info] Start training from score -1.613328
[LightGBM] [Info] Start training from score -1.611581
[LightGBM] [Info] Start training from score -1.609338
[LightGBM] [Info] Start training from score -1.610085
[LightGBM] [Info] Start training from score -1.602888
Accuracy: 0.9236691889021638
Classification Report:
                       precision    recall  f1-score   support

             Anxiety       0.92      0.92      0.92      3060
             Bipolar       0.92      0.93      0.93      3039
          Depression       0.91      0.85      0.88      3012
Personality disorder       0.95      0.99      0.97      3021
              Stress       0.90      0.92      0.91      2934

            

In [27]:
import joblib

joblib.dump(model, "mental_health_classifier_lgb.pkl")
joblib.dump(w2v_model, "word2vec_model.pkl")

['word2vec_model.pkl']

In [29]:
# Load Model
model = joblib.load("mental_health_classifier_lgb.pkl")
w2v_model = joblib.load("word2vec_model.pkl")

# Example tweet
new_text = ["i am happy"]
new_text_vectorized = np.array([vectorize_text(tweet, w2v_model) for tweet in new_text])

# Predict
predicted_label = model.predict(new_text_vectorized)
print("Predicted Mental Health Issue:", predicted_label[0])



Predicted Mental Health Issue: Depression


In [5]:
pip install lightgbm

Note: you may need to restart the kernel to use updated packages.


In [3]:
print(df['label'].unique())
print(f"Number of mental health issues detected: {df['label'].nunique()}")


['Anxiety' 'Depression' 'Stress' 'Bipolar' 'Personality disorder']
Number of mental health issues detected: 5
